In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
import torch
!pip install open3d 1>/dev/null
!pip install point-transformer-pytorch 1>/dev/null

In [36]:
from point_transformer_pytorch import PointTransformerLayer
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torch.optim as optim
import numpy as np

In [1]:
%cd /content/gdrive/MyDrive/Developer/coarse-to-fine-shapes
!git pull

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Developer/coarse-to-fine-shapes'
/content
fatal: not a git repository (or any of the parent directories): .git


## Load dataset

In [4]:
from datasets.shapenet_data_pc import ShapeNet15kPointClouds

In [14]:
shapenet_location = "/content/gdrive/MyDrive/Developer/ShapeNetCore_pointclouds/ShapeNetCore.v2.PC15k"
dataset = ShapeNet15kPointClouds(root_dir=shapenet_location, tr_sample_size=1024, te_sample_size=1024)

Total number of data:2832
Min number of points: (train)1024 (test)1024


## Define layer

In [10]:
# Example:
attn = PointTransformerLayer(
    dim = 128, # Feature dimension
    pos_mlp_hidden_dim = 64, # ?
    attn_mlp_hidden_mult = 4, # ?
    num_neighbors = 16          # only the 16 nearest neighbors would be attended to for each point
)

feats = torch.randn(1, 16, 128)
pos = torch.randn(1, 16, 3)
mask = torch.ones(1, 16).bool()

output = attn(feats, pos, mask = mask) # (1, 16, 128)
print(output.shape)

torch.Size([1, 16, 128])


In [20]:
single_layer = PointTransformerLayer(
    dim=3,
    pos_mlp_hidden_dim=64,
    attn_mlp_hidden_mult=4,
    num_neighbors=16
)

In [ ]:
AIRPLANE_INDEX = 0
airplane_input, airplane_gt = dataset[AIRPLANE_INDEX]['train_points'], dataset[AIRPLANE_INDEX]["test_points"]
print(airplane_input.shape, airplane_gt.shape)

In [43]:
feats=torch.randn(1,1024,3)
pos = torch.reshape(airplane_input, (1, *airplane_input.shape))
mask = torch.ones(1, 1024).bool()
print(feats.shape, pos.shape, mask.shape)
output = single_layer(feats, pos, mask=mask)

torch.Size([1, 1024, 3]) torch.Size([1, 1024, 3]) torch.Size([1, 1024])


## Overfit a single PointTransformer Layer on the airplane

In [24]:
from metrics.ChamferDistancePytorch.chamfer_python import distChamfer
def chamfer_distance(a, b):
    min_dist_a_to_b, min_dist_b_to_a, _, _ = distChamfer(a, b)
    return (min_dist_a_to_b.mean() + min_dist_b_to_a.mean()) / 2

In [30]:
print(chamfer_distance(torch.reshape(airplane_input, (1, *airplane_input.shape)), torch.reshape(airplane_gt, (1, *airplane_gt.shape))))

tensor(0.0061)


In [25]:
model = PointTransformerLayer(
    dim=3,
    pos_mlp_hidden_dim=64,
    attn_mlp_hidden_mult=4,
    num_neighbors=16
)

In [47]:
# TensorBoard Logging with model's name and current date
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
model_name = 'PointNet-SingleLayer'
log_dir = f"runs/{model_name}_{current_time}"
writer = SummaryWriter(log_dir=log_dir)

# Optimization
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.998)

# Losses
training_losses = []
val_losses = []

# Early Stopping (patience)
best_val_loss = np.inf
patience = 10
patience_counter = 0

input = torch.reshape(airplane_input, (1, *airplane_input.shape))
feats = torch.ones(*input.shape)
# output = torch.reshape(airplane_output, (1, *airplane_output.shape))

for epoch in range(100):
    total_train_loss = 0
    total_val_loss = 0

    # Train
    model.train()

    # Overfit on single pointcloud

    # input = airplane_input
    # feats = torch.ones(*input.shape)
    # output = airplane_gt

    optimizer.zero_grad()
    output = model(feats, input)
    loss = chamfer_distance(input, output)
    loss.backward()
    optimizer.step()

    total_train_loss += loss.item()

    epoch_train_loss = total_train_loss
    training_losses.append(epoch_train_loss)


    # # Validate
    # model.eval()
    # with torch.no_grad():
    #     for batch in val_dataloader:
    #         input = batch["train_points"].to("cuda").transpose(1,2)

    #         # Add noise to the input
    #         noised_input = add_random_noise(input)

    #         output = model(noised_input).to("cuda")
    #         val_loss = chamfer_distance(input, output)

    #         total_val_loss += val_loss.item()

    # epoch_val_loss = total_val_loss / len(val_dataloader)
    # val_losses.append(epoch_val_loss)

    writer.add_scalar('Loss/train', epoch_train_loss, epoch)
    # writer.add_scalar('Loss/val', epoch_val_loss, epoch)

    print(f"Epoch {epoch+1},  Training Loss: {epoch_train_loss:.4f}") #, Validation Loss: {epoch_val_loss:.4f}")

    # Check for improvement
    # if epoch_val_loss < best_val_loss:
    #     best_val_loss = epoch_val_loss
    #     patience_counter = 0  # reset the patience counter
    # else:
    #     patience_counter += 1
    #     print(f"No improvement in validation loss for {patience_counter} epochs.")

    # Break the loop if patience exceeded
    # if patience_counter >= patience:
    #     print("Stopping early due to no improvement in validation loss.")
    #     break

    # Save the model
    # model_save_path = f"output-models/pvcnn-refinement/PVCNN-Baseline-Refinement-e{epoch+1}.pth"
    # torch.save(model.state_dict(), model_save_path)


writer.close()

Epoch 1,  Training Loss: 2.5432
Epoch 2,  Training Loss: 2.5087
Epoch 3,  Training Loss: 2.4754
Epoch 4,  Training Loss: 2.4433
Epoch 5,  Training Loss: 2.4118
Epoch 6,  Training Loss: 2.3807
Epoch 7,  Training Loss: 2.3506
Epoch 8,  Training Loss: 2.3218
Epoch 9,  Training Loss: 2.2942
Epoch 10,  Training Loss: 2.2680
Epoch 11,  Training Loss: 2.2431
Epoch 12,  Training Loss: 2.2194
Epoch 13,  Training Loss: 2.1966
Epoch 14,  Training Loss: 2.1746
Epoch 15,  Training Loss: 2.1529
Epoch 16,  Training Loss: 2.1311
Epoch 17,  Training Loss: 2.1099
Epoch 18,  Training Loss: 2.0896
Epoch 19,  Training Loss: 2.0703
Epoch 20,  Training Loss: 2.0519
Epoch 21,  Training Loss: 2.0343
Epoch 22,  Training Loss: 2.0172
Epoch 23,  Training Loss: 2.0005
Epoch 24,  Training Loss: 1.9846
Epoch 25,  Training Loss: 1.9693
Epoch 26,  Training Loss: 1.9548
Epoch 27,  Training Loss: 1.9411


KeyboardInterrupt: 